<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [24]:
# Step 1
# Load and clean the data

import pandas as pd
columns = ['class', 'handicapped-infants', 'water-project-cost-sharing',
           'adoption-of-the-budget-resolution', 'physician-fee-freeze',
           'el-salvador-aid', 'religious-groups-in-schools',
           'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
           'mx-missile', 'immigration', 'synfuels-corporation-cutback',
           'education-spending', 'superfund-right-to-sue', 'crime',
           'duty-free-exports', 'export-administration-act-south-africa']
data_uri = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

df = pd.read_csv(data_uri, names=columns)
print(df.shape)
df.head()

(435, 17)


,class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [25]:
# df = df.replace({'?': float('nan'), 'y': 1, 'n': 0})
# STRETCH GOAL: refactor into a function
def boolify(df, t, f, n):
  lookup = { t:1, f:0, n:float('nan')}
  return df.replace(lookup)

df = boolify(df, 'y', 'n', '?')
df.head()

,class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [26]:
df.isnull().sum()

# Apparently we can tell our t-test what to do
# with NaN's; so I'll just leave them here

class                                       0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [28]:
# dems = df[df['class'] == 'democrat'].drop('class', axis=1)
# reps = df[df['class'] == 'republican'].drop('class', axis=1)
# STRETCH GOAL: refactor into a function
def byField(df, f, p):
  df = df[df[f] == p]
  return df.drop(f, axis=1)

dems = byField(df, 'class', 'democrat')
reps = byField(df, 'class', 'republican')
display(dems.head())
display(reps.head())

# Step 2
# Find an issue Dems support more than Reps,
# with p-value < 0.01

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
2,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [11]:
dems.describe()

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
reps.describe()

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
from scipy.stats import ttest_ind

# key = 'adoption-of-the-budget-resolution'
# ttest_ind(dems[key], reps[key], nan_policy='omit')
# STRETCH GOAL: refactor into a function
def quick_ttest(a, b, k):
  return ttest_ind(a[k], b[k], nan_policy='omit').pvalue
def polittest(k, ast):
  p = quick_ttest(dems, reps, k)
  print(f"p-value: {p}")
  assert(ast(p))
less1pct = lambda x: x < 0.01
great10pct = lambda x: x > 0.1

polittest('adoption-of-the-budget-resolution', less1pct)

# With a p-value of ~10^-77,
# DEMOCRATS SUPPORT `adoption of the budget resolution` MORE THAN REPUBLICANS

p-value: 2.0703402795404463e-77


In [37]:
# Step 3
# Find an issue Reps support more than Dem,
# with p-value < 0.01

polittest('education-spending', less1pct)

# With a p-value of ~10^-64,
# REPUBLICANS SUPPORT `education spending` MORE THAN DEMOCRATS

p-value: 1.8834203990450192e-64


In [38]:
# Step 4
# Find an issue Reps and Dems support
# with a p-value > 0.1

polittest('water-project-cost-sharing', great10pct)

# With a p-value of ~0.9,
# BOTH DEMOCRATS AND REPUBLICANS SUPPORT `water project cost sharing` EQUALLY

p-value: 0.9291556823993485


In [0]:
# STRETCHIEST OF STRETCH GOALS

# HYPOTHESIS TESTING W/ PERSONAL DATA

In [40]:
# load my dataset:
# game-level performance for QBs in the 2018 NFL season

qb_url = 'https://socratesdidnothingwrong.com/nfl/qbs.json';
df = pd.read_json(qb_url)
df = df[['Player', 'Week', 'Cmp', 'Att', 'Yds', 'TD', 'Int', 'Sk', 'Syds']]
df.head()

,Player,Week,Cmp,Att,Yds,TD,Int,Sk,Syds
0,Jared Goff,4,26,33,465,5,0,1,9
1,Ben Roethlisberger,10,22,25,328,5,0,1,6
2,Nate Sudfeld,17,1,1,22,1,0,0,0
3,Russell Wilson,8,14,17,248,3,0,2,11
4,Ryan Fitzpatrick,1,21,28,417,4,0,0,0


In [41]:
df.describe()

,Week,Cmp,Att,Yds,TD,Int,Sk,Syds
count,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000
mean,9.138648,19.816291,30.547660,224.686308,1.445407,0.724437,2.213172,14.727903
std,5.015066,8.651482,12.517137,105.535064,1.237809,0.906368,1.770113,12.761588
min,1.000000,0.000000,1.000000,-3.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,15.000000,24.000000,161.000000,0.000000,0.000000,1.000000,5.000000
50%,9.000000,21.000000,32.000000,233.000000,1.000000,0.000000,2.000000,13.000000
75%,14.000000,25.000000,39.000000,295.000000,2.000000,1.000000,3.000000,22.000000
max,17.000000,42.000000,62.000000,478.000000,6.000000,4.000000,11.000000,71.000000


In [42]:
# "any/a" is a kind of summary of how a QB did in a game
# let's calculate it, and add it as a column

# (it reduces all of these features into a single
# unit of measurement: "how many yards of progress
# will you get every time you throw the ball?")

def stats2anya(att, yds, td, ints, sk, syds):
  return (yds - syds + 20 * td - 45 * ints) / (att + sk)

df['any/a'] = df.apply(lambda r: stats2anya(r['Att'], r['Yds'], r['TD'], r['Int'], r['Sk'], r['Syds']), axis=1)
df.head()

,Player,Week,Cmp,Att,Yds,TD,Int,Sk,Syds,any/a
0,Jared Goff,4,26,33,465,5,0,1,9,16.352941
1,Ben Roethlisberger,10,22,25,328,5,0,1,6,16.230769
2,Nate Sudfeld,17,1,1,22,1,0,0,0,42.000000
3,Russell Wilson,8,14,17,248,3,0,2,11,15.631579
4,Ryan Fitzpatrick,1,21,28,417,4,0,0,0,17.750000


In [54]:
# now let's just get each QB's any/a throughout the season

df2 = df.pivot(index='Player', columns='Week', values='any/a')
df2.head()

# NaN means that the QB didn't play that week;
# we'll use nan_policy to ignore these in the t-tests

Week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Player,,,,,,,,,,,,,,,,,
A.J. McCarron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Rodgers,10.218750,5.934783,5.833333,6.119048,8.625000,9.081633,NaN,8.272727,6.590909,7.400000,10.028571,6.000000,4.843137,5.611111,4.042553,7.677966,2.333333
Alex Smith,8.696970,5.489796,10.750000,NaN,4.738095,5.025641,7.500000,6.187500,5.346939,6.333333,0.766667,NaN,NaN,NaN,NaN,NaN,NaN
Andrew Luck,5.454545,3.968750,3.833333,7.833333,5.416667,5.720930,10.260870,9.645161,NaN,10.344828,12.310345,7.973684,3.254545,8.860465,7.111111,7.250000,8.138889
Andy Dalton,7.466667,8.214286,3.958333,7.340909,6.375000,5.622222,3.548387,8.444444,NaN,2.458333,6.675676,4.411765,NaN,NaN,NaN,NaN,NaN


In [82]:
# HYPOTHESIS: Performance gets worse as the year goes on

def nfl_ttest(wk1, wk2):
  wkA = df2.iloc[:,wk1 - 1].tolist()
  wkB = df2.iloc[:,wk2 - 1].tolist()
  return ttest_ind(wkA, wkB, nan_policy='omit').pvalue

# Just for fun, let's compare every early week (1-8)
# with every late week (10-17) and see if there's
# ever any difference between the any/a's
ps = []
for n in range(1,9):
  for m in range(10,18):
    ps.append([n, m, nfl_ttest(n, m)])

# Now let's only look at comparisons that resulted
# in a "statistically significant" difference
list(filter(lambda r: r[2] < 0.05, ps))

[[2, 13, 0.02702418688588301],
 [2, 15, 0.014470386372224801],
 [4, 13, 0.03993212230990288],
 [4, 15, 0.023961351688971413],
 [8, 11, 0.023362085783480622],
 [8, 13, 0.009738473608601925],
 [8, 14, 0.042018951202061144],
 [8, 15, 0.007941798820724558],
 [8, 16, 0.03436681238860564]]

In [0]:
# ANALYSIS

# The core hypothesis, that play quality decreases as the year goes on,
# is not generally supported by the data (at least when looking at
# individual weeks vs each other); in most cases, the any/a's
# are indistinguishable between earlier or later

# HOWEVER

# The difference between the league's performance in Week 8 (right around
# the middle of the year) and later weeks (13-16) does seems to be significant

# In other words, it looks like we could say, "The league's performance is
# impacted by the time elapsed, if we only look at the second-half of the year"

# CAVEATS

# _Extremely small set of data_. On average, there will only be 30 observations
# per week, and this data only includes a single year of games



# TL;DR

# Is NFL performance impacted by lateness of the season?
# I WAS UNABLE TO REJECT THE NULL HYPOTHESIS :(